In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup

%run "catalog_support.py" 
showHeader('Holes in FracFocus')

In [ ]:
# fetch data set
df = fh.get_df(os.path.join(hndl.sandbox_dir,'workdf.parquet'))

In [ ]:
df = df[(df.date.dt.year>2010)]
df = df[df.in_std_filtered]


> ## Executive summary
> Although individual FracFocus disclosures may give the impression of a highly detailed and complete record of the chemicals injected into fracked wells, in reality, disclosures are far from transparent.  Members of the public looking to understand potential health and envirnomental consequences of frackings must come to grips with masked reports and  omissions from a full reporting.  These include proprietary claims and ambiguous data as well as missing chemicals because of reporting based on abbreviated lists. All of these sources of absent disclosure likely hide important dangerous materials. Although FracFocus is an important window into the industry's use of fracking chemicals, we must remember that it is far from a complete view.


[FracFocus](https://fracfocus.org/) is one of the biggest publicly available data sources of fracking chemical uses in the world.   It is cited by the fossil fuel industry as an example of its transparency and good stewardship. It covers over a decade of on-shore fracking operations in most of the US states where fracking is active.  This period includes at least part of the fracking boom that made the US a dominant oil and gas producer.  FracFocus includes millions of chemical records in over 175,000 disclosures.  


For people outside of the fossil-fuel industry, the motive to understand the uses of chemicals in fracking is simple: We want to understand the health and environmental consequences of the industry. **How useful is FracFocus data for this goal?**



# A naive expectation
One could reasonably expect that a chemical disclosure instrument like FracFocus would document all the chemicals used in a fracking operation, and in a manner that the public could use to understand the potential health and environmental impacts.  For example, at the least, if a Operator publishes a list of chemicals pumped into the ground 1,000 feet from an elementary school, like the six wells on a well pad near Mary Harris Elementary School in Tarrant County, Texas ...

In [ ]:
# Image(filename='./docs/Mary_Harris_Elementary.png', width=800)
from IPython.display import Image
Image(filename=os.path.join(hndl.nb_images_dir,'Mary_Harris_Elementary.png'), width=400)

([google map](https://www.google.com/maps/place/32%C2%B037'16.9%22N+97%C2%B022'58.4%22W/@32.6213565,-97.3850687,707m/data=!3m1!1e3!4m5!3m4!1s0x0:0xe79d0e9b033567aa!8m2!3d32.621352!4d-97.38288))

... we could expect the identity and quantity of all the chemicals to be clearly stated.   Something like the figure below:

Indeed, the detailed nature of individual FracFocus disclosures may give the impression that these data are a complete record. If you look at the 6 disclosures for the wells mentioned above using the FracFocus ["Find_a_Well' search feature](http://fracfocusdata.org/DisclosureSearch/Search.aspx) and these APINumbers: 42-439-34340, 42-439-34911, 42-439-37149, 42-439-37150, 42-439-37151, 42-439-37152, you will find detailed chemical information. 

But clearly, if any chemicals are left out of disclosures, our ability to understand fracking impacts is handicapped.

---
---
# Visible Holes
Once one begins examining FracFocus disclosures, the most obvious blow to that naive expectation is the extensive list of chemicals recorded in the disclosures that have their actual identities masked. The two major categories are Trade Secret claims and ambiguous records.

## Proprietary claims
The biggest class of these masked chemicals are the "proprietary claims," "trade secrets," and "confidential business information."  These claims are allowed by many state regulators and are used heavily in FracFocus.

Here is an example of these claims: the figure below is an excerpt of the disclosure for the Texas well 42-227-40269-00-00 by Diamondback E&P in August 2022 with the proprietary records marked with red.  

In [ ]:
Image(filename=os.path.join(hndl.nb_images_dir,'proprietary_disc_example.png'), width=600)

In [ ]:
c1 = df.bgCAS=='proprietary'
c2 = df.IngredientName.str.lower() == 'proprietary'
c3 = df.IngredientName == 'surfactant'
c4 = df.IngredientName == 'trade secret'
# print(f'len df: {len(df)}, len proprietary: {len(df[c1])}')
# print(df[c1][['bgCAS','IngredientName']].IngredientName.value_counts()[:10])
display(md(f"""The FracFocus website claims that proprietary records include the functional class of the chemical as shown in most records in the example
above. But even with a functional group, the lack of specific identification prevents us from understanding the actual risk of the chemical.
<br><br>Furthermore, there
are many records with essentially no indication of the material's class. For example, there are
**{len(df[c2]):,}** records named only "proprietary," **{len(df[c1&c4]):,}** records named only "trade secret" and **{len(df[c1&c3]):,}** records named only "surfactant."
<br><br>
Additionally, these masked chemicals are widespread: most disclosures report at least one record that is a proprietary claim, as illustrated below: """))

In [ ]:
Image(filename=os.path.join(hndl.nb_images_dir,'proprietary_pie.png'), width=500)

In [ ]:
display(md("""... as of Nov. 2022. Remarkably, there are [some disclosures](https://frackingchemicaldisclosure.wordpress.com/2020/04/24/why-can-companies-publish-disclosures-without-identifying-anything-but-water/) that hide **everything but water**. """ ))

c1 = df.bgCAS=='proprietary'
c2 = df.calcMass>0
t = df[c1 &c2].calcMass

display(md(f""" The percent of chemical records in FracFocus that are proprietary claims is **{round(len(df[c1])/len(df) *100,0)}%**. There are **{len(t):,}** proprietary
records that have non-zero mass. The median mass is {round(t.median(),0)} pounds.  That is a lot of unknown chemicals. <br> For more details into this set of records, 
see the [proprietary detail page.](./proprietary/analysis_proprietary.html)"""))
        

Are proprietary chemicals worth worrying about?  Of course, we can't know, because most likely, we will never learn the identity of these chemicals.  However, it is worth noting that when proprietary chemicals are occasionaly unmasked, they often contain dangerous materials.  In 2021, [EPA updated the TSCA "confidential" list](https://www.epa.gov/tsca-cbi/updates-confidential-status-chemicals-tsca-inventory) of materials and released the identity of about 300 chemicals whose identity had previously been protected.  Many of the chemicals on that list are so poorly studied, they have not even been assigned a CAS number. On that list, 11 chemicals are on EPA's PFAS master list.  

In [ ]:
Image(filename=os.path.join(hndl.nb_images_dir,'EPA_TSCA_non_confidential_PFAS.png'), width=800)

Again, we do not know if these have even been used in fracking operations; and that's the problem. A disclosure instrument is meant to provide tranparency.  Proprietary claims are antithetical to that.

## Ambiguous data
Another class of records like proprietary claims because they appear in disclosures are records that are simply not labeled or are labeled in a way that cannot be resolved to a single, specific chemical. That is, while they appear in the disclosure, companies do not give us enough information to understand their identity and therefore their toxicity.  

In [ ]:
c = df.bgCAS=='ambiguousID'
c1 = df.CASNumber=='MISSING'
c2 = df.PercentHFJob>0
c3 = df.PercentHFJob>75
c4 = df.CASNumber=='50-28-2'
display(md(f"""For example, there are **{len(df[c&c1&c2]):,}** records that have non-zero `PercentHFJob` 
but with no `CASNumber` given.  Among these records are  **{len(df[c&c1&c3]):,}** representing the carrier of the disclosure (% concentration > 75%).   While the `IngredientName` in that latter group implies some mix of water and other ingredients, there is no indication of what those other 
ingredients might be (see the "Produced Water" section below).  The mass of those records is **{th.round_sig(df[c&c1&c3].calcMass.sum())}** pounds (at least for those records for which mass is calculable). """))  

display(md(f"""Other ambiguous identification includes CAS codes that are either obvious dummy codes (e.g., '0000-00-0' or 'xxxx-xx-x')
or codes for chemicals that are clearly wrong.  For example, **{len(df[c4])}** disclosures report the proppant of the frack as '50-28-2' but that chemical is
**estradiol**, the human female hormone. Because this code is not typographically close to any FracFocus chemicals, this is likely not a mistake, but a stupid prank. 
<br><br> When we cannot resolve these records at Open-FF, we label them as "ambiguousID."  For more details into this set of records, 
see the [ambiguousID detail page.](./ambiguousID/analysis_ambiguousID.html) (We should also note that among the "ambiguousID" records are
clearly non-chemical lines in the disclosure. These will not have "quantity" data associated and are often formatting lines. They are
safely ignored.)
"""))

So, unlike the naive expectation, there are clearly holes that we can see:

In [ ]:
Image(filename=os.path.join(hndl.nb_images_dir,'Proprietary_and_Ambiguous.png'), width=350)

---
---
# Invisible Holes
## Abridged lists are the primary sources for disclosures
Unlike the Proprietary and Ambiguous classes that are essentially unidentified chemicals but we know they have been used, another set of chemicals are injected in fracking operations that are not mentioned in FracFocus disclosures.  We know they exist only from evidence outside of FracFocus.  Three classes that we know of are Safety Data Sheet omissions, water sources that contain more than water, and distillates.

## Safety Data Sheets
Much of the chemical information in FracFocus is based on products' Safety Data Sheets (SDS, sometimes called MSDS) that are compiled to comply with OSHA regulations. These documents, summarized [here by OSHA](https://www.osha.gov/sites/default/files/publications/OSHA3514.pdf) are primarily developed to inform workers who must work with these materials and to aid emergency response teams that might have to deal with spills or injuries. They are typically 3-15 pages and are usually found as PDF files. ([Click here for a quick example](https://ohiodnr.gov/static/documents/oil-gas/msds/Plexsurf+215E_Chemplex_SDS.pdf). Note that the format of SDS/MSDS has changed in the last decade; section numbers may be different than the current format.)

FracFocus and the companies submitting chemical disclosures have re-purposed Safety Data Sheets from their worker safety/emergency responder origins to serve as an ingredient list for disclosures by using Section 3 of the SDS, the composition of the product. Because manfacturers of chemical products used in fracking were already required to produce these SDS for each product, the SDS became a handy source of ingredients for public disclosure.  Below is a screenshot example of that ingredient list from a SDS for Plexsurf 251E by Chemplex:



In [ ]:
Image(filename=os.path.join(hndl.nb_images_dir,'Plexsurf_215E_ingredients.png'), width=600)

Note here that there are three hazardous ingredients listed and that their CAS numbers are listed. The percent listed is also required but may be given as a range to allow the manfacturer to mask exact recipes or to allow for variability in the manufacturing process. FracFocus (and state regulations) require that the *upper* level of the range be used in reporting.  

Clearly, this is an abridged list of what is in the product.  At most, these three ingredients comprise 46% of the mass in the product, and as little as 32%.  **What is in that other 54-68% of the product?**  Often, only the manufacturer knows. Even operating companies, while they are responsible for the completeness of the disclosure, may never know the ingredients of the products they use. See [Horwitt & Teklemichael, 2022](https://psr.org/wp-content/uploads/2022/07/chemical-makers-exemptions-from-fracking-chemical-disclosure-rules.pdf).

Are these non-disclosed chemicals a problem?  The SDS is not designed to be an exhaustive ingredient list unless all components are considered "hazardous."  Perhaps all of those non-identified ingredients are inert, perhaps just water.  But there are reasons to be suspicious of what is **not** on the list.  First, what may be considered non-harzardous in an occupational setting (the main purpose of an SDS) may be problematic from an environmental perspective.  Second, what is considered hazardous can change over time.  For example, the status of PFAS chemicals [have recently changed dramatically in the US](https://www.epa.gov/pfas). Although the EPA has recently disallowed the use of PFAS chemicals in pesticides, it has become clear that not only were they used in pesticides **as inert ingredients** (and therefore, not on the SDS of the products), they were used at [incredibly high concentrations](https://peer.org/substantial-pfas-contamination-found-in-pesticides/). By not listing all ingredients, we lose the ability to interpret impacts of past fracking events.

Could something like that happen with fracking chemicals? Considering that PFAS surfactants have been mentioned in multiple places (see [Horwitt & Gottlieb, 2022](https://psr.org/wp-content/uploads/2022/01/fracking-with-forever-chemicals-in-colorado.pdf)) as highly effective in fracking, and yet they appear rarely in FracFocus, it is reasonable to be suspicious that they may be used either as inert ingredients or hidden as proprietary ingredients.

In [ ]:
Image(filename=os.path.join(hndl.nb_images_dir,'SDS_example.png'), width=350)

Sidenote 1: While the use of SDS as a source of ingredients is only partial, you might think that at least it lends a predictability to FracFocus. That is, when you see a given product, you know the ingredients and their concentration is stable through the data set.  Unfortunately, that is not the case. In the table below are the 8 most frequent of at least 40 different variations of the Chemplex product "Plexsurf 240e."  In the `cas_per_tuple` column are the CAS numbers and concentrations (the maximum of the reported range) of each ingredient listed for that product in the disclosure.  `Size` is the number of disclosures that report that particular variation and `APINumber` are example wells from that set.

In [ ]:
Image(filename=os.path.join(hndl.nb_images_dir,'Plexsurf_240e_variation.png'), width=950)

All lines represent the same product and yet what is actually reported for the ingredients and their concentration for that product is far from stable across FracFocus.  For example, in the 4th line, the ingredient 71330-23-1 (an alkoxylated alcohol) is used at 15% (which for most of those 58 disclosures, was over 20,000 lbs of the chemical) - and yet that chemical doesn't even appear in the other 7 variations shown.  

From our preliminary exploration, such variation is more of the rule than the exception in FracFocus; when a TradeName is fairly common, the reporting of its chemicals will not be standardized.  Unfortunately for transparency sake, this type of exploration is not even possible with the "system approach" in which product names are explicitly dissociated from the chemical identification and quantity.  (User tip: search the [TradeName catalog](Open-FF_TradeNames.html) for other examples)

Sidenote 2: Safety Data Sheets can have a wealth of information about the chemicals that are actually disclosed on them.  There are several repositories of fracking SDS that are available free to the public: state regulator collections ([Ohio](https://ohiodnr.gov/discover-and-learn/safety-conservation/about-ODNR/oil-gas/sds), [Indiana](https://www.in.gov/dnr/oil-and-gas/oil-and-gas-activity/hydraulic-fracturing-in-indiana/dnr-hydraulic-fracturing-additives-msds/)) and company collections ([Halliburton (via Baroid)](https://www.baroididp.com/en/msds), [Baker Hughes (requires free account)](https://us.search.connect.bakerhughes.com/Pages/MSDSSearch.aspx).) However, many state SDS collections are old and many other products' SDS are effectively unavailable. Some companies that treat SDS as [confidential business information](https://www.slb.com/safety-data-sheets) and are therefore not available to the public.

## Produced water is not just water
In the vast majority of fracking disclosures, water is the primary carrier of proppants and other ingredients. This water is still an ingredient on the disclosure and often shows up with a concentration of 80% or more of the whole fracking fluid.  

The source of that water can come from nearby rivers or lakes, from municipal sources, from wells and from many other places.  One growing source is **produced water**, that is, water recovered from an active oil or gas well.  The volume of produced water can be many times the volume of the product and is a nuisance by-product of the industry.  This water can include "flow-back" materials from the original fracking slug.  Some research estimates that 30-70% of the fracking chemicals injected into a well returns to the surface in the produced water in the months following the injection.  While this produced water has been used extensively in "enhanced oil recovery," much of it is disposed in waste injection wells.  In recent years [there has been a push](https://www.texastribune.org/2022/12/19/texas-permian-basin-fracking-oil-wastewater-recycling/) to use it as part of the fracking carrier material in newer wells. 

The actual composition of produced water is usually hard to quantify.  Analysis of produced water is expensive and usually only tests for common markers such as mineral elements and known toxic materials (like BTEX).  Targeted analysis for specific molecules such as PFAS chemicals is rarely performed and, because it is targeted, can overlook materials that are important but not on the analyte list.

Nevertheless, when analysis of produced water is performed, it is very common to find a toxic cocktail, usually a combination of "natural" materials, such as hydrocarbons, radionuclides, and heavy metals as well as flow-back materials from previous fracks and the chemicals formed under the high pressure and temperature of the fracking pressure.  (For example, see [Stringfellow and Camarillo, 2019](https://escholarship.org/content/qt5b52w5nb/qt5b52w5nb.pdf?t=pqsbhu); Maguire-Boyle & Barron. Environ. Sci.: Processes Impacts, 2014,16, 2237-2248;  [Neff et. al, 2011](https://www.researchgate.net/publication/225911658_Produced_Water_Overview_of_Composition_Fates_and_Effects); [Leuk & Gonsior, 2017](http://dx.doi.org/10.1016/j.watres.2017.07.012).)

In [ ]:
df.TradeName.fillna('MISSING',inplace=True)
c1 = df.TradeName.str.lower().str.contains('produce')
c2 = df.bgCAS=='71-43-2'

gb = df[c1].groupby(['TradeName','DisclosureId'],as_index=False)['bgCAS'].count()
sing = len(gb[gb.bgCAS==1])
doub = len(gb[gb.bgCAS==2])

totBenz = df[c2].calcMass.sum()
prodBenz = df[c1&c2].calcMass.sum()
percProd = prodBenz/totBenz *100
display(md(f"""Produced water is therefore, much more than just water.  However, in many of the FracFocus disclosures 
that explicitly report the use of produced water, the reported composition is either just water alone ({sing} disclosures) or water
and some salt (usually sodium chloride or calcium chloride - {doub} disclosures)."""))

In [ ]:
from math import floor
display(md(f"""In one revealing set of about 120 FracFocus
disclosures in 2018-19 in Texas, a few other materials are included as ingredients of produced water, most notably [benzene](71-43-2/analysis_71-43-2.html) at a concentration of 0.1% of the 
produced water. This is at the high end of the range seen in published analysis, but not outside of it. Because of the large volume of produced water used in these fracks, this benzene release accounts 
for {floor(percProd)}% of all benzene mass reported in FracFocus.<br><br>"""))

Produced water, even once it has been through treatment, is hardly just water.  In a [California study](https://ww2.arb.ca.gov/sites/default/files/2020-07/CARB%20Oil%20Wastewater%20Emissions%20Final%20Report_05.11.2020_ADA.pdf) of produced water ponds (excerpt below), many aromatic analytes were found in the ~100 samples taken of the ponds. According to that study, produced water is sometimes taken from these ponds for resuse.

In [ ]:
 Image(filename=os.path.join(hndl.nb_images_dir,'CA_prod_water_ponds_analysis.png'), width=700)


Thus, based on the published analysis of produced water, it is likely that many of those reports of water-only "Produced Water" also included benzene and other chemicals of concern. 

But there is enough ambiguity in **all** disclosures that it is hard to know how much of the carrier water is fresh and how much contains contaminants.  Indeed, at the bottom of the all PDF disclosures we examined is a disclaimer: 
> **"Total Water Volume sources may include fresh water, produced water, and/or recycled water."**  

even though in the vast majority of disclosures, the carrier source is simply reported as 7732-18-5 (water).



In [ ]:
Image(filename=os.path.join(hndl.nb_images_dir,'Produced_water.png'), width=350)

## Distillates and other mixtures

In [ ]:
display(md(f"""A third group with abbreviated ingredient lists is a group of materials labeled by TSCA as "unknown, 
variable composition or biological" materials ([UVCBs](https://www.epa.gov/sites/default/files/2015-05/documents/uvcb.pdf)).
The UVCBs in fracking operations are mostly petroleum distillates.  """))

For distillates, the unknown and variable part of their composition begins with the source material: crude oil which is itself a UVCB and can contain hundreds of thousands of distinct chemicals.  [The distillation process](https://www.eia.gov/energyexplained/oil-and-petroleum-products/refining-crude-oil-the-refining-process.php) is able to separate different groups of materials based on their chemical properties (such as by volatility).  These fractions are often described as approximate because of huge variation in the source material, the specific conditions during the distillation process and variability across refineries.  For example there is a big difference between refinery-grade and laboratory-grade verisons of the same material.

The difficulty with these materials for users of FracFocus is that, while they are mixtures of many (often toxic) chemicals, they are almost always **represented by a single CAS number**.  For example, crude oil is listed simply as '8002-05-9'.  A common FracFocus material, "Heavy aromatic solvent naphtha (petroleum)," is similarly identified by a single CAS number: '64742-94-5'.  As its name suggests, one character that makes it distinct from many other distillates is that it has a high percentage of [aromatic compounds](https://en.wikipedia.org/wiki/Aromatic_compound) - that is, 64742-94-5 is designed to be heavily laden with chemicals like Benzene, Toluene, and Naphthalene. But because it is a UVCB, the precise mixture is unknown or at least variable and therefore not reported.

We can get a glimpse into some of the components of these UCVBs because some Safety Data Sheets for the UVCBs occasionally list those constituents.  For example, the Exxon SDS for their product "Solvent Naphtha H":

In [ ]:
Image(filename=os.path.join(hndl.nb_images_dir,'solvent_naphtha_H_SDS.png'), width=400)

 is **100% 64742-94-5**, but it also lists the components:

In [ ]:
Image(filename=os.path.join(hndl.nb_images_dir,'MSDS_47742X.png'), width=700)

Here is an excerpt from another product SDS from Monument Chemical, again 100% 64742-94-5, that also has the constituent chemicals listed:

In [ ]:
Image(filename=os.path.join(hndl.nb_images_dir,'Advasol_150_SDS.png'), width=600)

In [ ]:
Image(filename=os.path.join(hndl.nb_images_dir,'Advasol_150_comp.png'), width=700)

It is worth noting that, among all of the listed constituents, only 91-20-3 and 92-52-4 are ever directly reported in FracFocus.  Remember, this product is a solvent and therefore *designed* to be composed of aromatics.  Although we cannot know for sure, whenever 64742-94-5 is used, a significant portion of its mass is likely composed of these aromatics but they are hidden by the UVCB CAS number.

In [ ]:
Image(filename=os.path.join(hndl.nb_images_dir,'Distillate.png'), width=350)

#### The problem of UVCB for the public 
How can non-experts come to understand the risks of fracking chemicals?  We may look to **lists of problem chemicals** that have been assembled by experts after reviewing extensive detailed research.  Appropriate lists include the Clean Water Act lists, the Safe Drinking Water Act list, lists of known and suspect endocrine disrupters, and lists of carcinogens.  The problem for FracFocus users is that these lists depend on very well-defined chemical identification.  When a research paper is published that finds the toxicity of, say, naphthalene, researchers are reticent to apply those results to any other material, even if it is closely related, simply because toxicity can be highly dependent on specific molecular characteristics.   The consequence is that the UVCBs rarely show up on these lists of concern - even when they are very likely to contain those same toxic substances.

**How common are UVCBs in FracFocus?**  

In [ ]:
totbgcas = len(df.bgCAS.unique())-4 # ignore the non numeric ones.
uvcb = len(df[df.is_on_UVCB].bgCAS.unique())

c = df.in_std_filtered
c1 = df.bgCAS.isin(['7732-18-5','14808-60-7','1302-74-5','1302-76-7','1302-93-8',
                    '1318-16-7','1344-28-1','14464-46-1','308075-07-2','66402-68-4','proprietary',
                   'ambiguousID','sysAppMeta','conflictingID'])
totMass = df[c&~c1].calcMass.sum()
c2 = df.is_on_UVCB
uvcbmass = df[c&c2&~c1].calcMass.sum()
frac = uvcbmass/totMass
# print(f'tot: {totMass}, UVCB: {uvcb}, frac: {uvcb/totMass}')

display(md(f"""UVCBs form a fairly large group of materials in FracFocus disclosures: {uvcb} CAS numbers are UVCBs which is {uvcb/totbgcas:.0%} of the total.
And these materials represent a large fraction of the mass of chemicals injected.  If we look at just the mass of additives (that is, ignore water and proppants), we
find that UVCBs account for **{frac:.1%}** of those additives."""))

display(md(f"""(Tip: Type "UVCB" into the search bar of the [Chemical Index](Open-FF_Chemicals.html) to browse all UVCBs that have been reported in FracFocus)""")) 

**Unveiling those constituents: a simple model.**  If we make a few assumptions, we can start to see what might be missing from UVCB labelling. Let's look at just the naphthalene addition based on that one distillate (64742-94-5).  If we assume that the two SDS's above are good representations of 64742-94-5 used across FracFocus, we can say that the amount of naphthalene in every use of that product ranges from 0 to 14% of its mass.  

In [ ]:
dist_mass = df[df.bgCAS=='64742-94-5'].mass.sum()
napt_mass = df[df.bgCAS=='91-20-3'].mass.sum()
biph_mass = df[df.bgCAS=='92-52-4'].mass.sum()

display(md(f"""Considering that the currently reported mass of 64742-94-5 used is {th.round_sig(dist_mass,3)} pounds,
the range of added naphthalene due to that one distillate is **0 to {th.round_sig(dist_mass*0.14,3)} pounds**.  Compare that to the directly reported
naphthalene: {th.round_sig(napt_mass,3)} pounds. For the chemical biphenyl (92-52-4), the range would be **0 to {th.round_sig(dist_mass*0.08,3)} pounds** (0 to 8%). Compare that to the directly reported
biphenyl: {th.round_sig(biph_mass,3)} pounds (which has been reported in only {len(df[df.bgCAS=='92-52-4'])} records)."""))

# Conclusion
FracFocus has a huge amount of useful information about chemical use in the industry.  But users of that information must remember that it is partial.  The holes of FracFocus are substantial and are likely hiding important, dangerous materials.